In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import re

# Generate dataset

names = ["condensed_2010.json","condensed_2011.json","condensed_2012.json","condensed_2013.json",
         "condensed_2014.json", "condensed_2015.json","condensed_2016.json","condensed_2017.json","condensed_2018.json",
         "trump2019_26_03.json"]

data = pd.read_json("condensed_2009.json")
for element in names:
    data1 = pd.read_json(element)
    print(len(data1),element)
    data = data.append(data1,sort=True)
data = data.reset_index(drop=True)


#pd.to_datetime(stamps, format="%Y-%m-%d %H:%M:%S").sort_values()

data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data.to_json("orderedTwitterArchive.json")
data.to_csv("orderedTwitterArchive.csv")
data

# Exploration

In [ ]:
data = pd.read_json("orderedTwitterArchive.json")
data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="favorite_count", data=data) 

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax = sns.lineplot(x=data.index, y="retweet_count", data=data) 

In [ ]:
data["text"][2005]


In [ ]:
#check tags
ats = dict()
for idx,row in data.iterrows():
    l = re.findall("^@.[a-zA-Z0-9]*",row["text"])
    for el in l:
        if el not in ats:
            ats[el] = 1
        else:
            ats[el] = ats[el]+1
print(len(ats))
ats

In [ ]:
s = pd.Series(ats)
t = []
for el in s.index:
    if s[el] <= 2:
        t.append(el)
len(t)

In [ ]:
for el in t:
    s.drop(el,inplace=True)
s

In [ ]:
#sta sui 5 minuti, aggiunge una colonna per ogni tag con più di tot apparizioni (per migliorare il clustering?)
for tag in s.index:
    column = []
    for idx,row in data.iterrows():
        if tag in row["text"].split():
            column.append(1)
        else:
            column.append(0)
    data[tag] = pd.Series(column)

In [ ]:
data

In [ ]:
#check hastags
hashtags = dict()
for idx,row in data.iterrows():
    l = re.findall("^#[A-z]*",row["text"])
    for el in l:
        if el not in ats:
            hashtags[el] = 1
        else:
            hashtags[el] = hashtags[el]+1
hashtags
#usa pochissimo gli hashtag

In [ ]:
#remove hashtags, useless
for idx,row in data.iterrows():
    re.sub("^#[A-z]*","",row["text"])
#usa pochissimo gli hashtag

genereate new column with modded text, remove stopwords

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data["modded_text"] = data["text"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
data

In [ ]:
#remove punctuation
data['modded_text'] = data["modded_text"].str.replace("[^\w\s]",'')

#stemming
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()

stemmedDataset = pd.DataFrame()
data1 = data

stemDescriptions = []
for idx, row in data1.iterrows():
    desc = ""
    for el in row["text"].split():
        desc = desc + stemmer.stem(el) + " "
    stemDescriptions.append(desc)
    
data1["modded_text"] = stemDescriptions


#unique words
unique = pd.Series(' '.join(data1['modded_text']).split()).value_counts()
print(len(unique))
#devi rimuovere le cose con valore meno di tot e più di tot

In [ ]:
data1["modded_text"]

In [ ]:
unique.mean()


In [ ]:
unfreq = []
for el in unique.index:
    if unique[el] <= 5:
        unfreq.append(el)
len(unfreq)

# idee

NLP:
    - pos tagging per contare numero di aggettivi 
    - qualcosa per vedere se aggettivi sono positivi o negativi (direttamente sentiment analysis)?
    - rimuovere gli @ vs tenere solo i più frequenti 
    - rimuovere i link
    - pulire gli hashtag? 
    - word2vec distance per avere simili tweet assieme nello spazio vettoriale?
    - ridurre parole presenti? 
    - tfidf?
    
Processo:
    - clustering interno all'anno
    - clustering generale 
    - clustering con/senza sentiment meaning